<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study3/blob/main/MagnitudeGradientPruning1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow_model_optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 1.5 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
import tensorflow_model_optimization as tfmot

In [ ]:
# 데이터 로드 및 전처리 함수 정의
def load_data(base_dir, split, categories):
    X = []
    y = []
    split_dir = os.path.join(base_dir, split)
    for category in categories:
        category_dir = os.path.join(split_dir, category)
        for file in os.listdir(category_dir):
            file_path = os.path.join(category_dir, file)
            data = pd.read_csv(file_path, header=None).values
            data = pd.to_numeric(data.flatten(), errors='coerce').reshape(-1, data.shape[1])
            data = np.nan_to_num(data).astype('float32')  # NaN 값을 0으로 대체하고, float32로 변환
            X.append(data)
            y.append(category)
    return np.array(X), np.array(y)

# 기본 경로 및 카테고리 설정
base_dir = '/content/drive/MyDrive/ship_motor10'
categories = ['normal', 'fault_BB', 'fault_RI', 'fault_SM']

# 데이터 로드
X_train, y_train = load_data(base_dir, 'train', categories)
X_val, y_val = load_data(base_dir, 'validation', categories)
X_test, y_test = load_data(base_dir, 'test', categories)

# 데이터 차원 변경 (CNN 입력 형식에 맞게)
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# 원-핫 인코딩
y_train_categorical = tf.keras.utils.to_categorical(y_train_encoded)
y_val_categorical = tf.keras.utils.to_categorical(y_val_encoded)
y_test_categorical = tf.keras.utils.to_categorical(y_test_encoded)

In [ ]:
# 모델 정의
model = models.Sequential([
    layers.Conv1D(filters=64, kernel_size=16, strides=16, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    layers.MaxPooling1D(pool_size=2, strides=2),
    layers.Conv1D(filters=32, kernel_size=3, strides=1, activation='relu'),
    layers.Conv1D(filters=64, kernel_size=5, strides=1, activation='relu'),
    layers.Conv1D(filters=128, kernel_size=5, strides=1, activation='relu'),
    layers.MaxPooling1D(pool_size=2, strides=2),
    layers.Flatten(),
    layers.Dense(units=5000, activation='relu'),
    layers.Dense(units=1000, activation='relu'),
    layers.Dense(len(categories), activation='softmax')
])

# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 모델 훈련
history = model.fit(X_train, y_train_categorical, epochs=10, validation_data=(X_val, y_val_categorical), batch_size=32)

Epoch 1/10
394/394 [==============================] - 228s 575ms/step - loss: 0.9228 - accuracy: 0.6324 - val_loss: 0.4589 - val_accuracy: 0.8356
Epoch 2/10
394/394 [==============================] - 225s 571ms/step - loss: 0.4233 - accuracy: 0.8228 - val_loss: 0.0763 - val_accuracy: 1.0000
Epoch 3/10
394/394 [==============================] - 225s 572ms/step - loss: 0.1458 - accuracy: 0.9463 - val_loss: 0.0663 - val_accuracy: 0.9841
Epoch 4/10
394/394 [==============================] - 227s 576ms/step - loss: 0.0189 - accuracy: 0.9953 - val_loss: 0.0139 - val_accuracy: 0.9978
Epoch 5/10
394/394 [==============================] - 226s 573ms/step - loss: 0.0053 - accuracy: 0.9983 - val_loss: 0.0053 - val_accuracy: 0.9993
Epoch 6/10
394/394 [==============================] - 227s 575ms/step - loss: 4.9514e-04 - accuracy: 0.9999 - val_loss: 1.2045e-04 - val_accuracy: 1.0000
Epoch 7/10
394/394 [==============================] - 225s 570ms/step - loss: 0.0060 - accuracy: 0.9977 - val_loss: 

In [ ]:
# 프루닝 적용 코드
def magnitude_based_pruning(layer, sparsity):
    """매그니튜드 기반 프루닝"""
    if hasattr(layer, 'kernel'):
        weights, biases = layer.get_weights()
        threshold = np.percentile(np.abs(weights), sparsity * 100)
        pruned_weights = np.where(np.abs(weights) < threshold, 0, weights)
        layer.set_weights([pruned_weights, biases])

def gradient_sensitivity_pruning(layer, gradients, threshold_ratio):
    """변화도 기반 프루닝"""
    if hasattr(layer, 'kernel'):
        weights, biases = layer.get_weights()
        gradient_sensitivity = np.abs(gradients)
        threshold = np.percentile(gradient_sensitivity, threshold_ratio * 100)
        pruned_weights = np.where(gradient_sensitivity < threshold, 0, weights)
        layer.set_weights([pruned_weights, biases])

def combined_pruning(model, X_val, y_val, magnitude_sparsity=0.5, gradient_threshold=0.2):
    # 기울기 정보는 학습 과정에서 구해야 함
    with tf.GradientTape() as tape:
        logits = model(X_val, training=True)
        loss_value = tf.keras.losses.categorical_crossentropy(y_val, logits)
    gradients = tape.gradient(loss_value, model.trainable_variables)

    gradient_index = 0
    for layer in model.layers:
        if hasattr(layer, 'kernel'):
            # 1단계: 매그니튜드 기반 프루닝
            magnitude_based_pruning(layer, magnitude_sparsity)
            # 2단계: 변화도 기반 프루닝
            gradient_shape = layer.get_weights()[0].shape
            relevant_gradients = gradients[gradient_index].numpy().reshape(gradient_shape)
            gradient_sensitivity_pruning(layer, relevant_gradients, gradient_threshold)
            gradient_index += 2  # 각 레이어에 대해 커널과 바이어스가 있으므로 2씩 증가

            # 바이어스가 없는 경우에 대한 처리
            if len(layer.get_weights()) == 1:
                gradient_index -= 1  # 인덱스 보정

# 조합된 프루닝 수행
combined_pruning(model, X_val, y_val_categorical, magnitude_sparsity=0.9, gradient_threshold=0.8)

In [ ]:
# 스트립 프루닝 적용
model_stripped = tfmot.sparsity.keras.strip_pruning(model)

In [ ]:
# 프루닝 후 재학습 (파인 튜닝)
model_stripped.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history_finetune = model_stripped.fit(X_train, y_train_categorical, epochs=5, validation_data=(X_val, y_val_categorical), batch_size=32)

Epoch 1/5
394/394 [==============================] - 236s 596ms/step - loss: 1.2533 - accuracy: 0.4954 - val_loss: 1.2578 - val_accuracy: 0.5000
Epoch 2/5
394/394 [==============================] - 229s 582ms/step - loss: 1.2308 - accuracy: 0.5026 - val_loss: 1.0559 - val_accuracy: 0.5367
Epoch 3/5
394/394 [==============================] - 229s 582ms/step - loss: 0.3831 - accuracy: 0.8088 - val_loss: 0.1969 - val_accuracy: 0.9067
Epoch 4/5
394/394 [==============================] - 232s 590ms/step - loss: 0.2875 - accuracy: 0.8779 - val_loss: 0.1262 - val_accuracy: 0.9474
Epoch 5/5
394/394 [==============================] - 231s 587ms/step - loss: 0.1305 - accuracy: 0.9457 - val_loss: 0.2386 - val_accuracy: 0.9037


In [ ]:
# 모델 평가
test_loss, test_accuracy = model_stripped.evaluate(X_test, y_test_categorical)
print(f'Test accuracy after combined pruning, stripping, and fine-tuning: {test_accuracy}')

85/85 [==============================] - 4s 43ms/step - loss: 0.2035 - accuracy: 0.9281
Test accuracy after combined pruning, stripping, and fine-tuning: 0.9281481504440308


In [ ]:
# 프루닝 스트립 (추론 시 프루닝 마스크 제거)
model_for_export = tfmot.sparsity.keras.strip_pruning(model_stripped)

# 프루닝된 모델 저장
model_for_export.save('pruned_model.h5')

# 프루닝된 모델 사이즈 측정
pruned_model_size = os.path.getsize('pruned_model.h5') / (1024 * 1024)
print(f"Pruned Model Size: {pruned_model_size:.2f} MB")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Pruned Model Size: 463.71 MB


In [ ]:
# 스트립 프루닝 후 모델 저장
model_stripped.save('pruned_model_stripped.h5')

In [ ]:
def calculate_non_zero_weights(model):
    total_weights = 0
    non_zero_weights = 0
    for layer in model.layers:
        if hasattr(layer, 'kernel'):
            weights, biases = layer.get_weights()
            total_weights += np.prod(weights.shape) + np.prod(biases.shape)
            non_zero_weights += np.count_nonzero(weights) + np.count_nonzero(biases)
    return non_zero_weights, total_weights

non_zero_weights_after, total_weights_after = calculate_non_zero_weights(model_stripped)
print(f"After pruning: Non-zero weights = {non_zero_weights_after}, Total weights = {total_weights_after}")
print(f"Final non-zero weight ratio: {non_zero_weights_after / total_weights_after:.4f}")

After pruning: Non-zero weights = 40979060, Total weights = 121549684
Final non-zero weight ratio: 0.3371
